In [300]:
from datetime import date, datetime, timedelta
import pandas as pd
import pymysql
import numpy as np
from pykrx import stock
from time import sleep

ticker_count  = 0
commit_count  = 0
ticker_queue = []

# Connect to the database
connection = pymysql.connect(
    user='seokjin', 
    passwd='1234', 
    host='3.36.94.44', 
    db='stock', 
    charset='utf8'
)
stock_lsits = ['KOSPI', 'KOSDAQ', 'ETF']
cursor=connection.cursor()

for stock_list in stock_lsits:
    if stock_list != 'ETF':
        tickers = stock.get_market_ticker_list(market=stock_list)
    else:
        tickers = stock.get_etf_ticker_list()


    len_tickers = len(tickers)
    tickers = np.array(tickers).reshape(len_tickers,-1)
    ones = np.array([[stock_list,'Y',date.today().isoformat()]] * len_tickers)
    tickers = np.append(tickers, ones, axis = 1)


    commit_count = 0
    ticker_count = 0


    for ticker in tickers:
        commit_count+=1
        ticker_count+=1
        ticker_queue.append(ticker.tolist())

        if commit_count == 1000 or len_tickers == ticker_count:
            cols = "`,`".join([str(i) for i in ['code', 'code_gbn', 'check_item', 'create_date']])
            sql = "INSERT INTO `stock_kospi` (`" +cols + "`) VALUES (" + "%s,"*(ticker.shape[0]-1) + "%s)  ON DUPLICATE KEY UPDATE update_date ="+"'"+date.today().isoformat()+"'"
            cursor.executemany(sql,ticker_queue)

            connection.commit()
            commit_count=0
            ticker_queue = []


cursor.close()



In [321]:
from datetime import date, datetime, timedelta
import pandas as pd
import pymysql
import numpy as np
from pykrx import stock
from time import sleep

conn = pymysql.connect(
    user='seokjin', 
    passwd='1234', 
    host='3.36.94.44', 
    db='stock', 
    charset='utf8'
)
a = {}
stock_lsits = ['KOSPI', 'KOSDAQ', 'ETF']
try:
    curs = conn.cursor()
    # sql = "select * from stock_kospi a where not EXISTS (select 1 from (select DISTINCT code from daily_craw) b where a.code = b.code)and a.create_date = '2021-04-20' and a.code not in ('153130', '214980') "
    sel = "select * from stock_kospi where code ='072470'"
    curs.execute(sql)
    rs = curs.fetchall()
    df = pd.DataFrame(rs)

    for ticker in stock_lsits:
        a[ticker] = df[df[1]==ticker]

finally:
    conn.close()

stock_lsits = ['KOSPI', 'KOSDAQ', 'ETF']



for j in stock_lsits:
    for i in a[j][0]:
        if j != 'ETF':
            df = stock.get_market_ohlcv_by_date("19000101",'20210420', i)
        else :
            print(i)
            df = stock.get_etf_ohlcv_by_date("19000101", "20210420", i)

        df = change_df_to_array(df,i,j)
        insert_daily_craw(df)
   


KeyError: 1

In [266]:
def db_con():
    conn = pymysql.connect(
            user='seokjin', 
            passwd='1234', 
            host='3.36.94.44', 
            db='stock', 
            charset='utf8'
        )
    return conn

In [373]:
from datetime import date, datetime, timedelta
import pandas as pd
import pymysql
import numpy as np
from pykrx import stock
a = {}
# codes = ['049070','072470','036690','052330']
code = '153130'
date = date.today().isoformat()
df1 = stock.get_market_ohlcv_by_date("19000101", "20210420", code)
df1 = change_df_to_array(df1,code,'KOSPI')
print(len(df1))


# insert_daily_craw(df1)




2254


In [371]:
insert_daily_craw(df1)

In [366]:
df1[-1]

array(['1', '252670', '2021-04-20', '1950', '1955', '1915', '1920',
       '158550775'], dtype='<U21')

In [303]:
def change_df_to_array(df, ticker_code, stock_lsits):
    ticker_code_num = 0
    if stock_lsits == 'KOSPI':
        ticker_code_num = 1
    elif stock_lsits == 'KOSDAQ':
        ticker_code_num = 2
    elif stock_lsits == 'ETF':
        ticker_code_num = 3

    gbn_code = 0
    if ticker_code_num == 3:
        gbn_code = 1
    a = np.array([ticker_code_num, ticker_code]* len(df))
    a = np.array(a).reshape(len(df),-1)
    df['date'] = df.index.strftime('%Y-%m-%d')
    # 일자, 종가4, 시가1, 고가2, 저가3, 거래량5
    c =  np.array(df.values.tolist())[:,[-1,0+gbn_code,1+gbn_code,2+gbn_code,3+gbn_code,4+gbn_code]]
    a = np.append(a,c , axis = 1)

    return a

In [351]:
def insert_daily_craw(df):
    connection = db_con() 
    with connection.cursor() as cursor:

        len_df = len(df)
        ticker_count  = 0
        commit_count  = 0
        ticker_queue = []

        for insert_df in df:
            commit_count+=1
            ticker_count+=1
            ticker_queue.append(insert_df.tolist())

            if  len_df == ticker_count:
                cols = "`,`".join([str(i) for i in ['code_gbn', 'code', 'stock_date', 'stock_open','stock_high','stock_low','stock_close','volume']])

                sql = "INSERT INTO `daily_craw` (`" +cols + "`) VALUES (" + "%s,"*(df.shape[1]-1) + "%s)"
                cursor.executemany(sql,ticker_queue)

                connection.commit()
                commit_count=0
                ticker_queue = []
    connection.close()


In [249]:
from datetime import date, datetime, timedelta
import pandas as pd
import pymysql
import numpy as np
from pykrx import stock

code = '153130', '214980'
date = date.today().isoformat()
df1 = stock.get_etf_ohlcv_by_date("19000101", "20210420", code)
print(len(df1))
# df1.head(3)
ddd = change_df_to_array(df1,'122630','ETF')
insert_daily_craw(ddd)

2754


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
code_gbn int(1) not null comment '종목구분',
code varchar(10) NOT NULL COMMENT '종목코드' ,
stock_date date NOT NULL COMMENT '일자' ,
stock_open int(16)  NOT NULL COMMENT '시가' ,
stock_high int(16) NOT NULL  COMMENT '고가' ,
stock_low int(16)  NOT NULL COMMENT '저가' ,
stock_close int(16) NOT NULL  COMMENT '종가' ,
volume int(16) NOT NULL COMMENT '거래량',

In [258]:
conn = pymysql.connect(
    user='seokjin', 
    passwd='1234', 
    host='3.36.94.44', 
    db='stock', 
    charset='utf8',
    use_pure = True
)

TypeError: __init__() got an unexpected keyword argument 'use_pure'

In [263]:
conn = pymysql.connect(
    user='seokjin', 
    passwd='1234', 
    host='3.36.94.44', 
    db='stock', 
    charset='utf8'
)
try:
    curs = conn.cursor()
    sql = "show global variables like '%timeout%'"
    curs.execute(sql)
    rs = curs.fetchall()
    df = pd.DataFrame(rs)


finally:
    conn.close()

In [264]:
df

,0,1
0,connect_timeout,3600
1,delayed_insert_timeout,3600
2,have_statement_timeout,YES
3,innodb_flush_log_at_timeout,1
4,innodb_lock_wait_timeout,3600
5,innodb_rollback_on_timeout,OFF
6,interactive_timeout,28800
7,lock_wait_timeout,31536000
8,net_read_timeout,3600
9,net_write_timeout,3600


In [374]:
pip install pyupbitㅜ

     |████████████████████████████████| 88 kB 1.4 MB/s 
You should consider upgrading via the '/Users/seokjinlim/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [375]:
import pyupbit
print(pyupbit.Upbit)

<class 'pyupbit.exchange_api.Upbit'>


In [376]:
tickers = pyupbit.get_tickers()
print(tickers)

['KRW-BTC', 'KRW-ETH', 'BTC-ETH', 'BTC-LTC', 'BTC-XRP', 'BTC-ETC', 'BTC-OMG', 'BTC-CVC', 'BTC-DGB', 'BTC-SC', 'BTC-SNT', 'BTC-WAVES', 'BTC-NMR', 'BTC-XEM', 'BTC-LBC', 'BTC-QTUM', 'BTC-NXT', 'BTC-BAT', 'BTC-LSK', 'BTC-RDD', 'BTC-STEEM', 'BTC-DOGE', 'BTC-BNT', 'BTC-XLM', 'BTC-ARDR', 'BTC-KMD', 'BTC-ARK', 'BTC-ADX', 'BTC-SYS', 'BTC-ANT', 'BTC-STORJ', 'BTC-GRS', 'BTC-REP', 'BTC-RLC', 'BTC-EMC2', 'USDT-BTC', 'USDT-ETH', 'USDT-LTC', 'USDT-XRP', 'USDT-ETC', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC', 'KRW-XRP', 'KRW-ETC', 'KRW-OMG', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-KMD', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-REP', 'KRW-EMC2', 'KRW-ADA', 'BTC-ADA', 'BTC-MANA', 'USDT-OMG', 'KRW-SBD', 'BTC-SBD', 'BTC-RCN', 'KRW-POWR', 'BTC-POWR', 'KRW-BTG', 'USDT-ADA', 'BTC-DNT', 'BTC-IGNIS', 'BTC-ZRX', 'BTC-TRX', 'BTC-TUSD', 'BTC-LRC', 'KRW-ICX', 'KRW-EOS', 'BTC-DMT', 'USDT-TUSD', 'KRW-TRX', 'BTC-POLY', 'BTC-PRO', 'USDT-SC', 'USDT-TRX', 'KRW-SC', 'KRW-IGN